In [1]:
import pandas as pd

from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GroupShuffleSplit

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks


In [33]:
spotify = pd.read_csv('spotify.csv')

In [45]:
X=spotify.copy().dropna()
#X.head()

In [46]:
y = X.pop('track_popularity')
artists = X['track_artist']

features_num = ['danceability', 'energy', 'key', 'loudness', 'mode',
                'speechiness', 'acousticness', 'instrumentalness',
                'liveness', 'valence', 'tempo', 'duration_ms']
features_cat = ['playlist_genre']

preprocessor = make_column_transformer(
    (StandardScaler(), features_num),
    (OneHotEncoder(), features_cat),
)

# We'll do a "grouped" split to keep all of an artist's songs in one
# split or the other. This is to help prevent signal leakage.
def group_split(X, y, group, train_size=0.75):
    splitter = GroupShuffleSplit(train_size=train_size)
    train, test = next(splitter.split(X, y, groups=group))
    return (X.iloc[train], X.iloc[test], y.iloc[train], y.iloc[test])

X_train, X_valid, y_train, y_valid = group_split(X, y, artists)

X_train = preprocessor.fit_transform(X_train)
X_valid = preprocessor.transform(X_valid)
y_train = y_train / 100 # popularity is on a scale 0-100, so this rescales to 0-1.
y_valid = y_valid / 100

input_shape = [X_train.shape[1]]
print("Input shape: {}".format(input_shape))


Input shape: [18]


In [47]:
X_train

array([[ 0.64257806,  1.19515598,  0.17420341, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.13884592,  1.27805891, -1.20899107, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.43556485,  1.27253205,  0.45084231, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.91692147,  0.47666393, -1.48562997, ...,  0.        ,
         0.        ,  0.        ],
       [-0.86861838,  0.6701041 ,  0.17420341, ...,  0.        ,
         0.        ,  0.        ],
       [-0.35798579,  1.0182964 , -0.10243548, ...,  0.        ,
         0.        ,  0.        ]])

In [36]:
model = keras.Sequential([
    layers.Dense(1, input_shape=input_shape),
])
model.compile(
    optimizer='adam',
    loss='mae',
)
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=512,
    epochs=50,
    verbose=0, # suppress output since we'll plot the curves
)
history_df = pd.DataFrame(history.history)
#history_df.loc[0:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()));


Minimum Validation Loss: 0.2009


In [37]:
#history_df.loc[10:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()));

Minimum Validation Loss: 0.2009


In [38]:
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=input_shape),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])
model.compile(
    optimizer='adam',
    loss='mae',
)
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=512,
    epochs=50,
)
history_df = pd.DataFrame(history.history)
#history_df.loc[:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()));

Epoch 1/50
49/49 [==============================] - 1s 7ms/step - loss: 0.2336 - val_loss: 0.2124
Epoch 2/50
49/49 [==============================] - 0s 3ms/step - loss: 0.2016 - val_loss: 0.2067
Epoch 3/50
49/49 [==============================] - 0s 3ms/step - loss: 0.1957 - val_loss: 0.2053
Epoch 4/50
49/49 [==============================] - 0s 3ms/step - loss: 0.1932 - val_loss: 0.2029
Epoch 5/50
49/49 [==============================] - 0s 3ms/step - loss: 0.1910 - val_loss: 0.2041
Epoch 6/50
49/49 [==============================] - 0s 3ms/step - loss: 0.1889 - val_loss: 0.2021
Epoch 7/50
49/49 [==============================] - 0s 3ms/step - loss: 0.1881 - val_loss: 0.2018
Epoch 8/50
49/49 [==============================] - 0s 3ms/step - loss: 0.1869 - val_loss: 0.2024
Epoch 9/50
49/49 [==============================] - 0s 3ms/step - loss: 0.1855 - val_loss: 0.2010
Epoch 10/50
49/49 [==============================] - 0s 3ms/step - loss: 0.1848 - val_loss: 0.2024
Epoch 11/50
49/49 [

In [39]:
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=5, # how many epochs to wait before stopping
    restore_best_weights=True,
)

model = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=[11]),
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(1),
])
model.compile(
    optimizer='adam',
    loss='mae',
)

In [40]:
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=input_shape),
    layers.Dense(64, activation='relu'),    
    layers.Dense(1)
])
model.compile(
    optimizer='adam',
    loss='mae',
)
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=512,
    epochs=50,
    callbacks=[early_stopping]
)
history_df = pd.DataFrame(history.history)
#history_df.loc[:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()));

Epoch 1/50
49/49 [==============================] - 1s 7ms/step - loss: 0.2474 - val_loss: 0.2143
Epoch 2/50
49/49 [==============================] - 0s 3ms/step - loss: 0.2030 - val_loss: 0.2063
Epoch 3/50
49/49 [==============================] - 0s 3ms/step - loss: 0.1965 - val_loss: 0.2031
Epoch 4/50
49/49 [==============================] - 0s 4ms/step - loss: 0.1932 - val_loss: 0.2043
Epoch 5/50
49/49 [==============================] - 0s 3ms/step - loss: 0.1919 - val_loss: 0.2017
Epoch 6/50
49/49 [==============================] - 0s 3ms/step - loss: 0.1896 - val_loss: 0.2015
Epoch 7/50
49/49 [==============================] - 0s 3ms/step - loss: 0.1882 - val_loss: 0.2006
Epoch 8/50
49/49 [==============================] - 0s 4ms/step - loss: 0.1868 - val_loss: 0.2000
Epoch 9/50
49/49 [==============================] - 0s 4ms/step - loss: 0.1861 - val_loss: 0.2004
Epoch 10/50
49/49 [==============================] - 0s 3ms/step - loss: 0.1850 - val_loss: 0.2010
Epoch 11/50
49/49 [